성별 - 여자, 남자 <br>
연령대 - 10대, 20대 초, 20대 후, 30대 초, 30대 후 <br>
피부타입 - 건성, 지성, 중성, 복합성, 민감성 <br>
평점 - 짱짱!, 굿굿!, 쏘쏘!, 별로!, 최악! <br>
최신순, 오래된순, 좋아요 많은순, 좋아요 적은순

<= 100 : 2594개 -> 그냥 100개 크롤<br>
'> 100 : 375개 -> 조합 오지게해서 크롤 (이거 3개 중복 있음)

100~500/ : 289 <br>
500~1000/ 53 <br>
1000~/ : 33 <br>

---

machine 1대당 520개씩 할당

In [1]:
#### import modules
import pandas as pd
import re
import numpy as np
import pyautogui # 파이썬 매크로 모듈

from requests import get
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from time import sleep

In [9]:
#### load data
product_DB = pd.read_csv('SHIT.csv').iloc[150:,:]

In [32]:
product_DB.shape

(218, 7)

In [3]:
# path 조정
chrome_path = 'C:/Users/Ilhoon/Desktop/dev_util/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(chrome_path)

In [14]:
# 글로우픽 제품 리뷰
# 제품!
pid_product = [] # 제품 id
product_attribute = [] # 제품 속성
sales = [] # 판매처
product_desc = [] # 제품 설명
gl_num_rev = [] #글로우픽 리뷰 수

# 리뷰 데이터프레임
result_review = pd.DataFrame([])

# score 변환기
score_dict = {'worst': 1, 'bad': 2, 'soso': 3, 'good': 4, 'best':5}

# 필터셋
my_cases = pd.DataFrame([[5,t,6,s,r] for t in [2,3,4,5,6] for s in [2,3,4,5,6] for r in [1,2]])
my_cases.columns = ['drop_type','type','drop_score','score','recency']

In [15]:
# review 가져오는 함수...
def get_reviews(soup,product): # product는 product_id
    # initialize
    pid_rev = []
    score = [] # 평점
    age=[] # 나이
    skin_type=[] # 피부 타입
    date=[] # 날짜
    text=[] # 리뷰 텍스트
    
    # pid_rev
    pid_rev += [product] * len([rev.text for rev in soup.select('section.section-list-item > ul.review-list-wrap > li > div > p')])
    # score
    score += [score_dict[rate['class'][1][4:]] for s,rate in enumerate(soup.select('div > div > div > p > span.info > span.label > span')) if s % 2==0]
    #나이
    age += [ag.select('span.info > span.txt')[0].text.split()[0] for ag in soup.select('section.section-list-item > ul.review-list-wrap > li')]
    #피부타입
    skin_type += [sk.select('span.info > span.txt')[0].text.split()[2] for sk in soup.select('section.section-list-item > ul.review-list-wrap > li')]
    #날짜
    date += [dt.text for dt in soup.select('section.section-list-item > ul.review-list-wrap > li > span.date')]
    #리뷰텍스트
    text += [rev.text for rev in soup.select('section.section-list-item > ul.review-list-wrap > li > div > p')]
    
    result = pd.DataFrame({'pid' : pid_rev,'score':score ,'age' : age, 'skin_type' : skin_type, 'date' : date, 'text' : text})
    
    return result

In [16]:
def show_review(driver):
    # load all data
    for i in range(40):
        screenWidth, screenHeight = pyautogui.size()
        currentMouseX, currentMouseY = pyautogui.position()
        pyautogui.moveTo(1910, 1008)
        pyautogui.click()
        
    html = driver.page_source
    soup = bs(html,'lxml')
    return soup

In [17]:
def click_buttons(driver, drop_down, choose):
    """
    drop_down -- 5(피부타입)/ 6(평점)
    choose -- 2(건성/5점) 3(지성/4점) 4(중성/3점) 5(복합성/2점) 6(민감성/1점)
    """
    driver.find_element_by_xpath('//*[@id="gp-product-detail"]/div/ul[2]/li[2]/section/div/ul/li[' + str(drop_down) +']/div/button').click()
    driver.find_element_by_xpath('//*[@id="gp-product-detail"]/div/ul[2]/li[2]/section/div/ul/li[' + str(drop_down) +']/div/ul/li['+ str(choose) +']/div/button').click()
    
    return

In [18]:
def sort_recency(driver,choose):
    """
    1 - 최신순
    2 - 오래된순
    """
    driver.find_element_by_xpath('//*[@id="gp-product-detail"]/div/ul[2]/li[4]/section/div/div/button').click()
    driver.find_element_by_xpath('//*[@id="gp-product-detail"]/div/ul[2]/li[4]/section/div/div/ul/li['+ str(choose) +']/div/button').click()
    return
    

---

In [19]:
for ind,product in enumerate(product_DB.pid): # test101개 어퓨 리뷰와 1033개 미샤의 리뷰!!!
    driver.get('http://glowpick.com/product/'+str(product)+'')
    soup = show_review(driver)
    #---- 제품 정보 section----------------------
    # pid
    pid_product.append(product)
    
    # 속성
    product_attribute.append(','.join([attr.text for attr in soup.select('section > div > ul > li > ul > li')]))
    
    # sales
    if soup.select('section > div > ul > li.sale > div > ul > li') == []:
        sales.append('')
    else:
        sales.append(','.join([sal.text for sal in soup.select('section > div > ul > li.sale > div > ul > li')]))
    
    #product_desc
    product_desc.append(re.sub("\r+",",",re.sub("\n+","",soup.select('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > p')[0].text.strip())))
    
    #gl_num_rev
    gl_num_rev.append(int(re.sub('[^0-9]','',soup.select('div > span.joiner')[0].text)))
    
    
    #---- review section---------------------- 
    # 이쪽에 복잡한 logic 들어감...
    # 실제 총 리뷰 수가 맞는지 확인...
    total_rev = int(re.sub('[^0-9]','',soup.select('div > p.joiner')[0].text))

    if total_rev <= 100: # 만약 실제 총 리뷰가 100개 이하면...
        result_review = pd.concat((result_review,get_reviews(soup,product)),axis=0)
    else:
        for n in [1,2,3,4,5]:
            if int(soup.select('div > p.joiner')[n].text) <= 100: # 평점별 리뷰가 100개 이하면.... 그냥 평점으로만 필터 매기고 리뷰 가져오기
                # add filter
                sleep(5)
                click_buttons(driver,6,n+1) #5점

                # souping
                soup = show_review(driver)
                result_review = pd.concat((result_review,get_reviews(soup,product)),axis=0)
                result_review = result_review.drop_duplicates() # 중복 제거

                # reset #필터 제거
                driver.get('http://glowpick.com/product/'+str(product)+'') 
            else: # 평점별 리뷰가 100개 이상이면.... 피부타입/평점/최신순으로 필터 매겨서 갖고오기
                for j in range(len(my_cases.loc[my_cases.score == n+1,:])): 
                    dt = my_cases.loc[my_cases.score == n+1,:].iloc[j,:].drop_type
                    t = my_cases.loc[my_cases.score == n+1,:].iloc[j,:].type
                    ds = my_cases.loc[my_cases.score == n+1,:].iloc[j,:].drop_score
                    s = my_cases.loc[my_cases.score == n+1,:].iloc[j,:].score
                    r = my_cases.loc[my_cases.score == n+1,:].iloc[j,:].recency

                    sleep(5)
                    # add score filter
                    click_buttons(driver,dt,t)
                    sleep(5)
                    # add score filter
                    click_buttons(driver,ds,s)
                    sleep(5)
                    # sort by..
                    sort_recency(driver,r)

                    #soupping
                    soup = show_review(driver)
                    result_review = pd.concat((result_review,get_reviews(soup,product)),axis=0)
                    result_review = result_review.drop_duplicates() # 중복 제거

                    # reset #필터 제거
                    driver.get('http://glowpick.com/product/'+str(product)+'') 

    
    #--------------------------------------------
    print(ind)

0
1
2
3
4
5
6
7
8
9


WebDriverException: Message: unknown error: Element <button data-v-15a13080="" type="button" itemprop="query-input" class="ui-btn">...</button> is not clickable at point (444, 532). Other element would receive the click: <div data-v-7f247d50="" class="loading-wrap">...</div>
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.38.551601 (edb21f07fc70e9027c746edd3201443e011a61ed),platform=Windows NT 10.0.17134 x86_64)


In [16]:
len(pid_product)

35

In [27]:
pid_product[-1]

2714

In [24]:
len(gl_num_rev[:-1])

43

In [12]:
# product_DB
result_product_DB = pd.DataFrame({
    'pid' : pid_product[:-1],
    'product_attribute' : product_attribute[:-1],
    'sales' : sales[:-1],
    'product_desc' : product_desc[:-1],
    'gl_num_rev' : gl_num_rev[:-1]
})
len(result_product_DB)

34

In [13]:
result_product_DB.to_csv('product_db_4.csv',index=False)

In [21]:
result_review.to_csv('review_db_4.csv',index=False)

In [22]:
pid_product[-1]

102181

In [19]:
result_review.pid.values[-1]

627

In [12]:
pid_product

[907,
 3698,
 11075,
 42612,
 927,
 19055,
 73182,
 60053,
 12247,
 3543,
 24983,
 7970,
 52553,
 76999,
 77000,
 41847,
 3140,
 41766,
 33775,
 21664,
 11079,
 79804,
 81164,
 1768,
 5816,
 22302,
 64689,
 17154,
 1260,
 5007,
 9394,
 48801,
 20402,
 3209,
 37644,
 14325,
 41247,
 1800,
 34132,
 10717,
 13144,
 2373,
 575,
 2714]

In [16]:
len(pid_product)

44

In [14]:
len(result_review.pid.unique())

43

In [17]:
result_review.to_csv('review_db_3.csv',index=False)

In [21]:
result_review.loc[result_review.pid!=21798.0,:].to_csv('review_db_28_2.csv',index=False)

In [13]:
result_review

,pid,score,age,skin_type,date,text
0,907.0,4.0,27세,민감성,2개월 전,생각보다 평 안좋아서 깜짝.. 전 이거 바르고 클리오쿠션위에 발랐는데 피부 표현 잘...
1,907.0,2.0,24세,건성,2017.11.2,나 화장품이야아아!! 하는 냄새남.. 유분기가 엄청 많은편이라 얼굴이 건조한데도 번...
2,907.0,3.0,43세,지성,2016.8.13,가볍게 톤업시켜주는 베이스 용도로 쓰기에 좋은 제품. 단독사용하기엔 무리가 있고 이...
3,907.0,1.0,23세,중성,2016.7.3,살색크림이랑 흰색크림이 같이 나오는데 피부 상태 좋은 날은 그럭저럭 잘 받아요. 하...
4,907.0,4.0,27세,복합성,2016.3.9,구멍이 두개에서 두가지 나와야하는데 하나만 나와서 당황;; 그래서 살색 크림나오는것...
5,907.0,2.0,23세,복합성,2016.1.25,가벼움과 특이함에 혹해서 5개😨 구입햇지만...ㅎ 그거슨 나의인생똥템..ㅎ ㅠ손으로...
6,907.0,1.0,31세,복합성,2016.1.24,쓸애귀...각질써치 이거 한번쓰고 에스쁘아 화장품의 모든질을 의심될정도로 최악이여서...
7,907.0,2.0,28세,민감성,2016.1.24,서너번쓰고나서 안써서 남준 씨씨크림첨엔 신기해서 샀는데 좋지도않고 기름지고 ㅠㅠ
8,907.0,3.0,33세,복합성,2015.11.24,씨씨크림이라기보다는 그냥 톤업용 베이스?아니면 썬크림대용으로 쓸만한 정도인듯요.세일...
9,907.0,3.0,24세,건성,2015.9.23,세일할때 삿는대 음.. 걍... 집앞나갈때 좋다처음 바르면 겁나 허영허영한데 시간지...


In [ ]:
# to_csv
# productDB_over_1000.csv
# productDB_less_1000.csv 

---